In [1]:
#Imports
import keras
import keras.utils
from keras import utils as np_utils


C:\Users\Hamish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Function to load all the data
def load_all_data(filename):
    import numpy as np
    
    a = np.load(filename)
    d = (dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a))))

    return d

In [3]:
#Load the data
filename = ("alldata.npz")
X = load_all_data(filename)['data1arr_0']
y = load_all_data(filename)['data1arr_1']

IDs = load_all_data(filename)['data1arr_2']

In [4]:
#Split Data into Train_Test
from  sklearn.model_selection import GroupShuffleSplit
import numpy as np
test_size = 0.2
gss = GroupShuffleSplit(n_splits = 1, test_size = 0.2)

for train,test in gss.split(X, y, IDs):
    X_train = X[train]
    y_train = y[train]
    IDs_train = IDs[train]
    
    X_test = X[test]
    y_test = y[test]
    IDs_test = IDs[test]

fileoutname = 'train_test_data'
np.savez(fileoutname,X_train, y_train, X_test, y_test,IDs_train,IDs_test)

In [5]:
#Split Data in Train2_Dev
gss = GroupShuffleSplit(1, test_size = 0.2)

for train,test in gss.split(X, y, IDs):
    X_train2 = X[train]
    y_train2 = y[train]
    IDs_train = IDs[train]
    
    X_dev = X[test]
    y_dev = y[test]
    IDs_test = IDs[test]

fileoutname = 'train_dev_data'
np.savez(fileoutname,X_train, y_train, X_test, y_test, IDs_train, IDs_test)

In [6]:
#Add dimensions for X in CNN and Convert y to Categorical
X_train2 = np.expand_dims(X_train2,axis=0)
y_train2 = keras.utils.to_categorical(y_train2,num_classes=2)
X_dev = np.expand_dims(X_test,axis=0)
y_dev = keras.utils.to_categorical(y_test,num_classes=2)
X = np.expand_dims(X,axis=0)  

In [7]:
#Build the CNN
def  simpleCNN(self, units = 10):
    
    import keras
    from keras.layers import Dense
    from keras.layers import Conv2D
    from keras.layers import Flatten
    from keras.models import Model, Input
    
    inp =  Input(shape = self.shape[1:], name='inp')
    #layer 1
    x = Conv2D(units, kernel_size=(1,1), strides = (1,1), activation='relu', data_format='channels_last')(inp)  
    #layer 2
    x = Conv2D(units, kernel_size=(2,2), strides = (1,1), activation='relu', data_format='channels_last')(x)  
    #layer 3 
    x = Flatten()(x)
    #layer4
    out = Dense(2, activation='softmax',name='out')(x)
    
    model = Model(inputs = inp, outputs = out)
    
    return model


In [8]:
model = simpleCNN(X)

In [9]:
#Fit Data to Model
from keras.optimizers import Adamax
adamax = Adamax(lr=3e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0); #3e-4; 2e-3 is a default.
model.compile(optimizer=adamax, loss='categorical_crossentropy', metrics=['acc'])
model.fit(X_train2, y_train2, epochs=20, batch_size=32, verbose = 1, validation_data = (X_dev, y_dev))

ValueError: Error when checking input: expected inp to have shape (11459, 26, 60) but got array with shape (8743, 26, 60)